In [1]:
import pandas as pd
import datetime ## How many total missing values do we have?
from os import listdir # os is a module for interacting with the OS
from os.path import isfile, join # to verify file object, and concatenate paths
import glob # to find pathnames matching a specific pattern
import re # regular expressions :)

In [2]:
#import data set
df = pd.read_csv("./archive.csv")

In [3]:
#display sum of missing values
df_nulls = df.isnull().sum()
df_nulls 

Year                      0
Category                  0
Prize                     0
Motivation               88
Prize Share               0
Laureate ID               0
Laureate Type             0
Full Name                 0
Birth Date               29
Birth City               28
Birth Country            26
Sex                      26
Organization Name       247
Organization City       253
Organization Country    253
Death Date              352
Death City              370
Death Country           364
dtype: int64

It seems like the Birth City is missing 28 values. Let's see which of the winners have those missing birth city values

In [4]:
#copy the full name, birth city, birth date, laureate type, organization city, organization name, and sex
newdf = df[['Full Name','Birth City','Birth Date', 'Laureate Type','Organization Country','Organization City','Organization Name','Sex']].copy()
newdf
#display the rows in which the Birth City is null
d1 = newdf[(newdf['Birth City'].isnull())]
d1

,Full Name,Birth City,Birth Date,Laureate Type,Organization Country,Organization City,Organization Name,Sex
24,Institut de droit international (Institute of ...,NaN,NaN,Organization,NaN,NaN,NaN,NaN
61,Bureau international permanent de la Paix (Per...,NaN,NaN,Organization,NaN,NaN,NaN,NaN
90,Comité international de la Croix Rouge (Intern...,NaN,NaN,Organization,NaN,NaN,NaN,NaN
206,Office international Nansen pour les Réfugiés ...,NaN,NaN,Organization,NaN,NaN,NaN,NaN
222,Comité international de la Croix Rouge (Intern...,NaN,NaN,Organization,NaN,NaN,NaN,NaN
244,Friends Service Council (The Quakers),NaN,NaN,Organization,NaN,NaN,NaN,NaN
245,American Friends Service Committee (The Quakers),NaN,NaN,Organization,NaN,NaN,NaN,NaN
295,Office of the United Nations High Commissioner...,NaN,NaN,Organization,NaN,NaN,NaN,NaN
365,Comité international de la Croix Rouge (Intern...,NaN,NaN,Organization,NaN,NaN,NaN,NaN
366,Ligue des Sociétés de la Croix-Rouge (League o...,NaN,NaN,Organization,NaN,NaN,NaN,NaN


<h1>Observation</h1>
<span>As you might have noticed most of the Birth City missing values are coming from Laureate Type organization. Not only is the Birth City missing, but also the birth date, sex, organization name, country, and city. This makes sense, as winning as an organization means that you are a group of people which might come from different univeristies (organization), birth cities, organization countries, cities, and are of differenet sexes. Besides, each of them might have a different birth date</span>

<span>Moreover, for the laureate type organization, the full name contains the name of the organization, not the name of the person. This is an indication that they are a group not an individual person who won the Nobel prize</span>

In [5]:
organizationCount = newdf[(newdf['Laureate Type']=="Organization")].count() 
# from the above line we can observe the number of existing birth city when the laureate type is organization

#check the number of rows with missing Birth City and laureate type of individual
countBirthIndividual = newdf[(newdf['Birth City'].isnull()) &(newdf['Laureate Type']=="Individual")].count()
display(organizationCount,countBirthIndividual)


Full Name               30
Birth City               4
Birth Date               4
Laureate Type           30
Organization Country     0
Organization City        0
Organization Name        0
Sex                      4
dtype: int64

Full Name               2
Birth City              0
Birth Date              2
Laureate Type           2
Organization Country    0
Organization City       0
Organization Name       0
Sex                     2
dtype: int64

<h1>Checking number of organization laureate type</h1>
<span>What we want to observe is the number of organization laureate type, and those with missing birth city, organization name, organization country, organization city, birth date, and sex</span>

In [6]:
missingOrganization = newdf[(newdf['Laureate Type']=="Organization")&(newdf['Organization Name'].isnull())&(newdf['Organization City'].isnull())&(newdf['Organization Country'].isnull())&(newdf['Birth City'].isnull())&(newdf['Sex'].isnull())&(newdf['Birth Date'].isnull())].count()
missingOrganization

Full Name               26
Birth City               0
Birth Date               0
Laureate Type           26
Organization Country     0
Organization City        0
Organization Name        0
Sex                      0
dtype: int64

<h1> Analyzing the above result </h1>
The number of rows of laureate type organization which contain missing organization info, sex, birth date, and birth city are 26/30 rows. Likewise, only 4 rows contained birth city even without checking on the organization info (Organization name, city, and country). Now let's check the rows which contain the birth city and are of laurete type orgainzation.

In [7]:
#checking the 4 rows which contain birth date, birth city, sex, and are of type organization
laureate_org_expection = newdf[(newdf['Laureate Type'] == 'Organization') & (newdf['Sex'].notnull()) & (newdf['Birth Date'].notnull())& (newdf['Birth City'].notnull())]
laureate_org_expection

,Full Name,Birth City,Birth Date,Laureate Type,Organization Country,Organization City,Organization Name,Sex
452,Le Duc Tho,Nam Ha province,1911-10-14,Organization,NaN,NaN,NaN,Male
520,Mother Teresa,Uskup (Skopje),1910-08-26,Organization,NaN,NaN,NaN,Female
620,The 14th Dalai Lama (Tenzin Gyatso),Taktser,1935-07-06,Organization,NaN,NaN,NaN,Male
640,Aung San Suu Kyi,Rangoon (Yangon),1945-06-19,Organization,NaN,NaN,NaN,Female


<h1> Why do sexes, birth dates, and borth cities exist above? </h1>
<span>It seems like not only the birth city, birth date, and sex exist, but also, the full names are names of actual people and not a group. This might indicate that those people were part of an organization, or this data is mistakenly categorized as of laureate type <span style ="font-style:italic">Organization<span> </span>

<h1>Checking if we can drop rows of <span style ="font-style:italic">Laureate Type:</span> <span style ="font-style:italic">Organization<span> </h1>
<span>Let's check how many rows of type organization exist. If they are less or equal to 3% perecent of the data, we will choose to drop any column of type organization and focus our study on individuals</span>

In [8]:
# How many total missing values do we have?
# shape returns the dimentionality of a dataframe (rows and columns)
# how many rows exist in the data set
dfNumofRows = df.shape[0]
# how many rows exist with type organization
laureateOrganization_df = newdf[(newdf['Laureate Type']=="Organization")]
orgNumofRows= laureateOrganization_df.shape[0]
#calculate the perecentage of organization data with respect to the whole data set
perc_drop= (orgNumofRows/dfNumofRows)*100
perc_drop

3.0959752321981426

Since the perecentage of the rows to be dropped, rows which carry laureate type <span style ="font-style:italic">Organization</span> is approximately 3, we will choose to drop those rows.

In [9]:
#drop the rows of type organization
df = df[df['Laureate Type'] != "Organization"] 

In [10]:
df

,Year,Category,Prize,Motivation,Prize Share,Laureate ID,Laureate Type,Full Name,Birth Date,Birth City,Birth Country,Sex,Organization Name,Organization City,Organization Country,Death Date,Death City,Death Country
0,1901,Chemistry,The Nobel Prize in Chemistry 1901,"""in recognition of the extraordinary services ...",1/1,160,Individual,Jacobus Henricus van 't Hoff,1852-08-30,Rotterdam,Netherlands,Male,Berlin University,Berlin,Germany,1911-03-01,Berlin,Germany
1,1901,Literature,The Nobel Prize in Literature 1901,"""in special recognition of his poetic composit...",1/1,569,Individual,Sully Prudhomme,1839-03-16,Paris,France,Male,NaN,NaN,NaN,1907-09-07,Châtenay,France
2,1901,Medicine,The Nobel Prize in Physiology or Medicine 1901,"""for his work on serum therapy, especially its...",1/1,293,Individual,Emil Adolf von Behring,1854-03-15,Hansdorf (Lawice),Prussia (Poland),Male,Marburg University,Marburg,Germany,1917-03-31,Marburg,Germany
3,1901,Peace,The Nobel Peace Prize 1901,NaN,1/2,462,Individual,Jean Henry Dunant,1828-05-08,Geneva,Switzerland,Male,NaN,NaN,NaN,1910-10-30,Heiden,Switzerland
4,1901,Peace,The Nobel Peace Prize 1901,NaN,1/2,463,Individual,Frédéric Passy,1822-05-20,Paris,France,Male,NaN,NaN,NaN,1912-06-12,Paris,France
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
964,2016,Medicine,The Nobel Prize in Physiology or Medicine 2016,"""for his discoveries of mechanisms for autophagy""",1/1,927,Individual,Yoshinori Ohsumi,1945-02-09,Fukuoka,Japan,Male,Tokyo Institute of Technology,Tokyo,Japan,NaN,NaN,NaN
965,2016,Peace,The Nobel Peace Prize 2016,"""for his resolute efforts to bring the country...",1/1,934,Individual,Juan Manuel Santos,1951-08-10,Bogotá,Colombia,Male,NaN,NaN,NaN,NaN,NaN,NaN
966,2016,Physics,The Nobel Prize in Physics 2016,"""for theoretical discoveries of topological ph...",1/2,928,Individual,David J. Thouless,1934-09-21,Bearsden,United Kingdom,Male,University of Washington,"Seattle, WA",United States of America,NaN,NaN,NaN
967,2016,Physics,The Nobel Prize in Physics 2016,"""for theoretical discoveries of topological ph...",1/4,929,Individual,F. Duncan M. Haldane,1951-09-14,London,United Kingdom,Male,Princeton University,"Princeton, NJ",United States of America,NaN,NaN,NaN


In [11]:
# we can drop the Laureate Type column since it is the same for all rows (records).
df['Laureate Type'].nunique()

1

<h1>Handling Duplicates</h1>
We can now check for duplicate rows (rows with the same <span style="font-style:italic">Laureate ID</span>), and drop them.

In [12]:
# # Check the  duplicate rows based on the laureate id:
# duplicateRowsDF = df[df.duplicated(['Laureate ID'])] 
# print("The rows with the same laureate ID are:", duplicateRowsDF['Full Name'], sep='\n')

# #drop those cols which are repeated
# #df.drop_duplicates('Laureate ID')

<h1>Check missing <span style="font-style:italic">Birth Dates</h1>
<body>Now that the data contains indivdual laureates only, let's check how many of them is missing the birth date, given that we have dropped the organization laureates which had missing birth dates as they were groups, and dropped duplicates.</body>

In [14]:
newdf = df[['Full Name','Birth City','Laureate ID']].copy()
newdf[newdf["Birth City"].isnull()]

,Full Name,Birth City,Laureate ID
750,Sir Vidiadhar Surajprasad Naipaul,NaN,747
880,Liu Xiaobo,NaN,855


In [ ]:
 # Select all duplicate rows based on one column
# duplicateRowsDF = df[df.duplicated(['Laureate ID'])]
#  df
# #print("Duplicate Rows based on a single column are:", duplicateRowsDF, sep='\n')